In [1]:
#We read in the libraries.
library(tidyverse)
library(lme4)
library(sjPlot)
library(stargazer)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 4.3.2"

Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Learn more about sjPlot with 'browseVignettes("sjPlot")'.


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R packag

In [3]:
#We read in the nested file.
df <- read.csv("41_nested_data.csv", row.names=1)

In [ ]:
#We estimate an empty intercept model, predicting transparency index by the clusters nested author, nested section, and outlet.
empty <- lmer(transparency_index ~ 1 + (1|nested_author) + (1|nested_section) + (1|news_source), data = df, control = lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 1000000)))

In [5]:
#wr print the output.
summary(empty)

Linear mixed model fit by REML ['lmerMod']
Formula: transparency_index ~ 1 + (1 | nested_author) + (1 | nested_section) +  
    (1 | news_source)
   Data: df
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 1e+06))

REML criterion at convergence: -40936.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5229 -0.6513 -0.0995  0.5087  5.3736 

Random effects:
 Groups         Name        Variance Std.Dev.
 nested_author  (Intercept) 0.004437 0.06661 
 nested_section (Intercept) 0.002540 0.05040 
 news_source    (Intercept) 0.008664 0.09308 
 Residual                   0.009027 0.09501 
Number of obs: 23546, groups:  
nested_author, 2124; nested_section, 327; news_source, 6

Fixed effects:
            Estimate Std. Error t value
(Intercept)  0.23450    0.03825   6.131

In [7]:
library(modelsummary)

Warning message:
"package 'modelsummary' was built under R version 4.3.1"


In [8]:
#We transform the output to a latex format.
modelsummary(empty, "latex")

Warning message:
"To compile a LaTeX document with this table, the following commands must be placed in the document preamble:

\usepackage{booktabs}
\usepackage{siunitx}
\newcolumntype{d}{S[
    input-open-uncertainty=,
    input-close-uncertainty=,
    parse-numbers = false,
    table-align-text-pre=false,
    table-align-text-post=false
 ]}

To disable `siunitx` and prevent `modelsummary` from wrapping numeric entries in `\num{}`, call:

options("modelsummary_format_numeric_latex" = "plain")
 This warning appears once per session."


\begin{table}
\centering
\begin{tabular}[t]{lc}
\toprule
  & (1)\\
\midrule
(Intercept) & \num{0.235}\\
 & (\num{0.038})\\
SD (Intercept nested\_author) & \num{0.067}\\
SD (Intercept nested\_section) & \num{0.050}\\
SD (Intercept news\_source) & \num{0.093}\\
SD (Observations) & \num{0.095}\\
\midrule
Num.Obs. & \num{23546}\\
R2 Marg. & \num{0.000}\\
R2 Cond. & \num{0.634}\\
AIC & \num{-40926.3}\\
BIC & \num{-40885.9}\\
ICC & \num{0.6}\\
RMSE & \num{0.09}\\
\bottomrule
\end{tabular}
\end{table}

In [6]:
#We extract the overall random intercept variation.
random_var <- VarCorr(empty)$nested_author + VarCorr(empty)$nested_section + VarCorr(empty)$news_source

In [7]:
#We extract the residual variance.
residual_var <- attr(VarCorr(empty), "sc")^2

In [8]:
#We add the residual and random intercept variance to get the overall random variance.
total_var <- random_var + residual_var

In [9]:
#We calculate the VPC for nested author, which we will also do for section and outlet.
round(VarCorr(empty)$nested_author / total_var, 2)

,(Intercept)
(Intercept),0.18


In [10]:
round(VarCorr(empty)$nested_section / total_var, 2)

,(Intercept)
(Intercept),0.1


In [11]:
round(VarCorr(empty)$news_source / total_var, 2)

,(Intercept)
(Intercept),0.35


In [12]:
#We calculate the total VPC.
round(random_var / total_var, 2)

,(Intercept)
(Intercept),0.63
